In [43]:
using Zygote
function tbf90(I01p, T, incAng, k, a, g, n, eps, dz)
    cos1 = cos(incAng / 180.0 * π)
    Jup0 = zeros(Float64, n + 1)
    Jd0 = zeros(Float64, n + 1)
    Jup = Zygote.Buffer(Jup0)
    Jd = Zygote.Buffer(Jd0)
    intDo0 = zeros(Float64, n + 1)
    intUp0 = zeros(Float64, n + 1)
    intDo = Zygote.Buffer(intDo0)
    intUp = Zygote.Buffer(intUp0)

    for k1 in 1:n+1
        Jup[k1] = (1 - a[k1]) * T[k1] + a[k1] * (0.5 * (I01p[2*k1-1] + I01p[2*k1+1]) * g[k1] * cos1 + I01p[2*k1])
        Jd[k1] = (1 - a[k1]) * T[k1] + a[k1] * (-0.5 * (I01p[2*k1-1] + I01p[2*k1+1]) * g[k1] * cos1 + I01p[2*k1])
    end

    intDo[1] = k[1] * dz / 2
    intUp[n+1] = k[n+1] * dz / 2

    for k1 in 1:n
        intUp[n+1-k1] = intUp[n-k1+2] + 0.5 * (k[n+1-k1] + k[n+1-k1+1]) * dz
    end

    for k1 in 2:n+1
        intDo[k1] = intDo[k1-1] + 0.5 * (k[k1] + k[k1-1]) * dz
    end
    sumJD = Float64(0)
    sumJU = Float64(0)
    #sumJD = 
    #sumJU = Zygote.Buffer(sumJU0)

    for i in 1:n+1
        sumJD += Jd[i] * exp(-intDo[i] / cos1) * k[i] * dz / cos1
        sumJU += Jup[i] * exp(-intUp[i] / cos1) * k[i] * dz / cos1
    end

    Tb = (1 - eps) * exp(-intUp[1] / cos1) * sumJD + sumJU + eps * exp(-intUp[1] / cos1) * T[1]
    return Tb
end

function SetEddington1D(T, k, a, g, n, eps, dz, Ts)
    #Abig = Zygote.Buffer(zeros(Float64, 2*n+3, 2*n+3))
    Abig0 = zeros(Float64, 2*n+3, 2*n+3)
    B0 = zeros(Float64, 2*n+3)
    Abig = Zygote.Buffer(Abig0)
    B = Zygote.Buffer(B0)
    for i in 0:n
        Abig[2*i+2, 2*i+2] = 3 * k[i+1] * (1 - a[i+1]) * dz
        Abig[2*i+2, 2*i+3] = 1
        Abig[2*i+2, 2*i+1] = -1
        B[2*i+2] = 3 * k[i+1] * (1 - a[i+1]) * T[i+1] * dz

        if 2*i+3 < 2*n+3
            km = 0.5 * (k[i+1] + k[i+2])
            am = 0.5 * (a[i+1] + a[i+2])
            gm = 0.5 * (g[i+1] + g[i+2])
            Abig[2*i+3, 2*i+3] = km * (1 - am * gm) * dz
            Abig[2*i+3, 2*i+4] = 1
            Abig[2*i+3, 2*i+2] = -1
            B[2*i+3] = 0.0
        end
    end

    Abig[2*n+3, 2*n+2] = 1.0
    Abig[2*n+3, 2*n+3] = -1.0 / 3.0
    Abig[2*n+3, 2*n+1] = -1.0 / 3.0
    B[2*n+3] = 2.7

    Abig[1, 1] = (2 - eps) / (3 * eps)
    Abig[1, 2] = 1.0
    Abig[1, 3] = (2 - eps) / (3 * eps)
    B[1] = Ts
    return Abig, B
end


SetEddington1D (generic function with 1 method)

In [44]:

function tbwrapper(kext1, scat1, asym1, temp1, dz, eps, n, incAng)
    temp_mid = 0.5 * (temp1[1:end-1] + temp1[2:end])
    Abig_py, B_py = SetEddington1D(kext1, scat1, asym1, temp_mid, n, eps, dz, temp1[1])
    I01p_py = torch.linalg.solve(Abig_py, B_py)
    Tb_py = tbf90_py(I01p_py, temp_mid, incAng, kext1, scat1, asym1, n, eps, dz)
    return Tb_py
end

tbwrapper (generic function with 1 method)

In [35]:
using Zygote
kext1 = Float64[0.5, 0.6, 0.7, 0.8]
scat1 = Float64[0.3, 0.4, 0.5, 0.6]
asym1 = Float64[0.1, 0.2, 0.3, 0.4]
temp1 = Float64[300.0, 310.0, 320.0, 330.0,330.0]
dz = Float64(0.25)
eps = Float64(0.9)
n = length(kext1) - 1
incAng = Float64(45.0)



45.0

In [45]:
#tb_forward=tbwrapper(kext1, scat1, asym1, temp1, dz, eps, n, incAng)
function tbwrapper(kext1, scat1, asym1, temp1, dz, eps, n, incAng)
    temp_mid = 0.5 * (temp1[1:end-1] + temp1[2:end])
    A,B=SetEddington1D(temp_mid, kext1, scat1, asym1, n, eps, dz, temp1[1]);
    I01p=A\B;
    Tb_jl=tbf90(I01p, temp1, incAng, kext1, scat1, asym1, n, eps, dz);
    return Tb_jl
end


tbwrapper (generic function with 1 method)

In [46]:
tb_forward=tbwrapper(kext1, scat1, asym1, temp1, dz, eps, n, incAng)
grad = Zygote.gradient((kext1, scat1, asym1, temp1, eps) -> tbwrapper(kext1, scat1, asym1, temp1, dz, eps, n, incAng), kext1, scat1, asym1, temp1, eps)



LoadError: Mutating arrays is not supported -- called setindex!(Vector{Float64}, ...)
This error occurs when you ask Zygote to differentiate operations that change
the elements of arrays in place (e.g. setting values with x .= ...)

Possible fixes:
- avoid mutating operations (preferred)
- or read the documentation and solutions for this error
  https://fluxml.ai/Zygote.jl/latest/limitations


In [31]:
#incAng, k, a, g, n, eps, dz
#calculate of the gradient of Tb with respect to kext1, scat1, asym1, temp1, eps, I01p
temp_mid = 0.5 * (temp1[1:end-1] + temp1[2:end])
A,B=SetEddington1D(temp_mid, kext1, scat1, asym1, n, eps, dz, temp1[1]);
I01p=A\B;
Tb_jl=tbf90(I01p, temp1, incAng, kext1, scat1, asym1, n, eps, dz);
println(typeof(I01p))
println(typeof(temp1))
println(typeof(incAng))
println(typeof(kext1))
println(typeof(scat1))
println(typeof(asym1))
println(typeof(eps))
grad = Zygote.gradient((I01p, temp1, kext1, scat1, asym1, eps) -> tbf90(I01p, temp1, incAng, kext1, scat1, asym1, n, eps, dz), I01p, temp1, kext1, scat1, asym1, eps)

Vector{Float64}
Vector{Float64}
Float64
Vector{Float64}
Vector{Float64}
Vector{Float64}
Float64


([0.0007421181520280422, 0.025220422555734454, 0.00372028866073439, 0.04768987514482673, 0.01137589400917483, 0.08563218592155354, 0.028752316454698786, 0.15072216100757765, 0.020354592954230307], [0.4509583908793771, 0.07153481271724008, 0.08563218592155354, 0.10048144067171845, 0.0], [18.81322314838421, -3.9899222007523854, -5.860094151808049, -9.127123189671082], [-6.8641281521109505, -11.523337430528308, -19.16202519308969, -31.871893045153705], [1.6030832475467638, 4.057293728796677, 9.630742497056513, 21.77468111473663], 68.54896327058623)

In [2]:
using PyCall